In [1]:
import re 
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
df = pd.read_csv('/Users/maximascientia/Documents/Data/Reviews.csv')
print(df.shape)
df.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,ReviewText,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
# Convert to list
df['ReviewText'] = df['ReviewText'].apply(str)
data = df.ReviewText.values.tolist()
data_words = list(data)
print(data_words[:1])

['Absolutely wonderful - silky and sexy and comfortable']


In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words ('english')
stop_words.extend(['from', 'to', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'na', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'br', 'sub', 'mech','non'])

In [7]:
texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in data_words]

In [8]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts]

In [9]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

2022-02-17 17:01:19,819 : INFO : collecting all words and their counts
2022-02-17 17:01:19,820 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-02-17 17:01:20,046 : INFO : PROGRESS: at sentence #10000, processed 252375 words and 133032 word types
2022-02-17 17:01:20,291 : INFO : PROGRESS: at sentence #20000, processed 508877 words and 223342 word types
2022-02-17 17:01:20,378 : INFO : collected 250397 token types (unigram + bigrams) from a corpus of 597343 words and 23486 sentences
2022-02-17 17:01:20,378 : INFO : merged Phrases<250397 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
2022-02-17 17:01:20,379 : INFO : Phrases lifecycle event {'msg': 'built Phrases<250397 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000> in 0.56s', 'datetime': '2022-02-17T17:01:20.379870', 'gensim': '4.1.2', 'python': '3.8.2 (default, Mar 26 2020, 10:43:30) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'c

In [10]:
# Remove rare and common tokens.
from gensim import corpora

# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2022-02-17 17:01:26,216 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-17 17:01:26,460 : INFO : adding document #10000 to Dictionary(8691 unique tokens: ['absolutely', 'comfortable', 'sexy', 'silky', 'wonderful']...)
2022-02-17 17:01:26,703 : INFO : adding document #20000 to Dictionary(11655 unique tokens: ['absolutely', 'comfortable', 'sexy', 'silky', 'wonderful']...)
2022-02-17 17:01:26,789 : INFO : built Dictionary(12457 unique tokens: ['absolutely', 'comfortable', 'sexy', 'silky', 'wonderful']...) from 23486 documents (total 626454 corpus positions)
2022-02-17 17:01:26,789 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12457 unique tokens: ['absolutely', 'comfortable', 'sexy', 'silky', 'wonderful']...) from 23486 documents (total 626454 corpus positions)", 'datetime': '2022-02-17T17:01:26.789715', 'gensim': '4.1.2', 'python': '3.8.2 (default, Mar 26 2020, 10:43:30) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'macOS-10.16-x86_64-i386

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 2422
Number of documents: 23486


In [16]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-02-17 17:21:51,258 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2022-02-17 17:21:51,260 : INFO : using serial LDA version on this node
2022-02-17 17:21:51,262 : INFO : running online (multi-pass) LDA training, 3 topics, 20 passes over the supplied corpus of 23486 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-02-17 17:21:51,265 : INFO : PROGRESS: pass 0, at document #2000/23486
2022-02-17 17:21:53,486 : INFO : optimized alpha [0.146823, 0.13000703, 0.14894865]
2022-02-17 17:21:53,487 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:21:53,488 : INFO : topic #0 (0.147): 0.028*"dress" + 0.021*"fit" + 0.019*"love" + 0.017*"size" + 0.014*"like" + 0.013*"great" + 0.011*"small" + 0.010*"color" + 0.010*"wear" + 0.009*"fabric"
2022-02-17 17:21:53,489 : INFO : topic #1 (0.130): 0.020*"top" + 0.016*"fit" + 

2022-02-17 17:22:00,645 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:22:00,645 : INFO : topic #0 (0.149): 0.042*"dress" + 0.022*"fit" + 0.018*"love" + 0.016*"wear" + 0.016*"size" + 0.012*"great" + 0.011*"like" + 0.010*"perfect" + 0.008*"small" + 0.008*"look"
2022-02-17 17:22:00,646 : INFO : topic #1 (0.134): 0.022*"top" + 0.019*"love" + 0.018*"color" + 0.014*"look" + 0.014*"fit" + 0.014*"great" + 0.012*"size" + 0.011*"like" + 0.010*"shirt" + 0.009*"wear"
2022-02-17 17:22:00,647 : INFO : topic #2 (0.130): 0.019*"size" + 0.015*"dress" + 0.014*"like" + 0.014*"fit" + 0.014*"top" + 0.012*"look" + 0.010*"color" + 0.010*"ordered" + 0.009*"love" + 0.009*"one"
2022-02-17 17:22:00,647 : INFO : topic diff=0.199104, rho=0.333333
2022-02-17 17:22:00,653 : INFO : PROGRESS: pass 0, at document #20000/23486
2022-02-17 17:22:01,394 : INFO : optimized alpha [0.14997587, 0.13570973, 0.1334175]
2022-02-17 17:22:01,395 : INFO : merging changes from 2000 documen

2022-02-17 17:22:06,036 : INFO : topic diff=0.149488, rho=0.269749
2022-02-17 17:22:06,038 : INFO : PROGRESS: pass 1, at document #12000/23486
2022-02-17 17:22:06,665 : INFO : optimized alpha [0.15451984, 0.1534632, 0.1483406]
2022-02-17 17:22:06,666 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:22:06,667 : INFO : topic #0 (0.155): 0.048*"dress" + 0.024*"fit" + 0.018*"love" + 0.017*"wear" + 0.016*"size" + 0.012*"great" + 0.012*"perfect" + 0.009*"like" + 0.009*"flattering" + 0.009*"comfortable"
2022-02-17 17:22:06,667 : INFO : topic #1 (0.153): 0.021*"top" + 0.020*"love" + 0.020*"color" + 0.015*"great" + 0.014*"look" + 0.014*"fit" + 0.012*"like" + 0.011*"wear" + 0.011*"shirt" + 0.011*"size"
2022-02-17 17:22:06,668 : INFO : topic #2 (0.148): 0.021*"size" + 0.015*"fit" + 0.015*"dress" + 0.015*"top" + 0.015*"like" + 0.012*"small" + 0.012*"look" + 0.011*"ordered" + 0.009*"really" + 0.009*"fabric"
2022-02-17 17:22:06,668 : INFO : topic diff=0.1534

2022-02-17 17:22:11,373 : INFO : topic #2 (0.174): 0.022*"size" + 0.016*"top" + 0.015*"like" + 0.015*"dress" + 0.015*"fit" + 0.013*"small" + 0.012*"look" + 0.011*"ordered" + 0.010*"fabric" + 0.009*"really"
2022-02-17 17:22:11,374 : INFO : topic diff=0.138409, rho=0.260440
2022-02-17 17:22:11,376 : INFO : PROGRESS: pass 2, at document #6000/23486
2022-02-17 17:22:11,948 : INFO : optimized alpha [0.16316287, 0.1762216, 0.17588077]
2022-02-17 17:22:11,948 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:22:11,949 : INFO : topic #0 (0.163): 0.048*"dress" + 0.024*"fit" + 0.019*"love" + 0.018*"wear" + 0.015*"size" + 0.013*"great" + 0.012*"perfect" + 0.010*"comfortable" + 0.009*"flattering" + 0.008*"like"
2022-02-17 17:22:11,950 : INFO : topic #1 (0.176): 0.021*"color" + 0.021*"love" + 0.020*"top" + 0.016*"great" + 0.014*"look" + 0.014*"fit" + 0.012*"wear" + 0.012*"like" + 0.011*"soft" + 0.011*"shirt"
2022-02-17 17:22:11,951 : INFO : topic #2 (0.176):

2022-02-17 17:22:16,517 : INFO : topic #1 (0.196): 0.022*"color" + 0.021*"love" + 0.020*"top" + 0.016*"great" + 0.014*"look" + 0.013*"fit" + 0.012*"like" + 0.012*"wear" + 0.012*"soft" + 0.011*"sweater"
2022-02-17 17:22:16,517 : INFO : topic #2 (0.204): 0.022*"size" + 0.017*"top" + 0.016*"dress" + 0.015*"like" + 0.015*"fit" + 0.014*"small" + 0.012*"look" + 0.011*"ordered" + 0.010*"fabric" + 0.009*"back"
2022-02-17 17:22:16,518 : INFO : topic diff=0.131389, rho=0.260440
2022-02-17 17:22:16,519 : INFO : PROGRESS: pass 2, at document #23486/23486
2022-02-17 17:22:16,942 : INFO : optimized alpha [0.17502338, 0.19543308, 0.20942707]
2022-02-17 17:22:16,942 : INFO : merging changes from 1486 documents into a model of 23486 documents
2022-02-17 17:22:16,943 : INFO : topic #0 (0.175): 0.046*"dress" + 0.024*"fit" + 0.020*"love" + 0.017*"wear" + 0.015*"size" + 0.013*"great" + 0.013*"perfect" + 0.010*"comfortable" + 0.010*"flattering" + 0.009*"perfectly"
2022-02-17 17:22:16,944 : INFO : topic #1 (

...

2022-02-17 17:23:24,740 : INFO : topic #1 (1.413): 0.033*"color" + 0.029*"love" + 0.023*"great" + 0.017*"top" + 0.017*"soft" + 0.015*"sweater" + 0.015*"look" + 0.013*"wear" + 0.012*"shirt" + 0.009*"fit"
2022-02-17 17:23:24,741 : INFO : topic #2 (1.951): 0.028*"size" + 0.024*"dress" + 0.019*"fit" + 0.017*"like" + 0.016*"top" + 0.015*"small" + 0.012*"ordered" + 0.012*"look" + 0.010*"fabric" + 0.009*"large"
2022-02-17 17:23:24,741 : INFO : topic diff=0.065924, rho=0.186524
2022-02-17 17:23:24,743 : INFO : PROGRESS: pass 16, at document #20000/23486
2022-02-17 17:23:25,030 : INFO : optimized alpha [0.7263723, 1.4129163, 1.9854972]
2022-02-17 17:23:25,031 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:23:25,032 : INFO : topic #0 (0.726): 0.043*"dress" + 0.026*"fit" + 0.024*"wear" + 0.023*"pant" + 0.022*"love" + 0.022*"perfect" + 0.021*"jean" + 0.017*"comfortable" + 0.014*"great" + 0.014*"perfectly"
2022-02-17 17:23:25,032 : INFO : topic #1 (1.413)

2022-02-17 17:23:27,205 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:23:27,206 : INFO : topic #0 (0.744): 0.043*"dress" + 0.027*"fit" + 0.025*"wear" + 0.023*"jean" + 0.022*"love" + 0.022*"perfect" + 0.021*"pant" + 0.017*"comfortable" + 0.015*"perfectly" + 0.014*"great"
2022-02-17 17:23:27,207 : INFO : topic #1 (1.451): 0.033*"color" + 0.029*"love" + 0.023*"great" + 0.017*"top" + 0.017*"soft" + 0.015*"look" + 0.014*"wear" + 0.013*"sweater" + 0.013*"shirt" + 0.009*"fit"
2022-02-17 17:23:27,207 : INFO : topic #2 (2.010): 0.028*"size" + 0.025*"dress" + 0.020*"fit" + 0.017*"like" + 0.017*"top" + 0.016*"small" + 0.012*"ordered" + 0.012*"look" + 0.010*"fabric" + 0.009*"really"
2022-02-17 17:23:27,208 : INFO : topic diff=0.065603, rho=0.183361
2022-02-17 17:23:27,210 : INFO : PROGRESS: pass 17, at document #14000/23486
2022-02-17 17:23:27,485 : INFO : optimized alpha [0.7472701, 1.4470139, 2.019764]
2022-02-17 17:23:27,485 : INFO : merging changes 

2022-02-17 17:23:29,363 : INFO : topic diff=0.061397, rho=0.180354
2022-02-17 17:23:29,365 : INFO : PROGRESS: pass 18, at document #6000/23486
2022-02-17 17:23:29,632 : INFO : optimized alpha [0.7580526, 1.4848412, 2.0738947]
2022-02-17 17:23:29,633 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:23:29,633 : INFO : topic #0 (0.758): 0.042*"dress" + 0.026*"fit" + 0.025*"wear" + 0.023*"love" + 0.022*"jean" + 0.022*"perfect" + 0.021*"pant" + 0.018*"comfortable" + 0.015*"perfectly" + 0.015*"great"
2022-02-17 17:23:29,634 : INFO : topic #1 (1.485): 0.034*"color" + 0.029*"love" + 0.023*"great" + 0.017*"top" + 0.017*"soft" + 0.015*"look" + 0.014*"sweater" + 0.013*"wear" + 0.013*"shirt" + 0.009*"bought"
2022-02-17 17:23:29,635 : INFO : topic #2 (2.074): 0.028*"size" + 0.024*"dress" + 0.020*"fit" + 0.017*"like" + 0.016*"top" + 0.016*"small" + 0.012*"ordered" + 0.012*"look" + 0.011*"fabric" + 0.009*"large"
2022-02-17 17:23:29,635 : INFO : topic diff=0.0

2022-02-17 17:23:31,820 : INFO : topic #2 (2.113): 0.028*"size" + 0.024*"dress" + 0.019*"fit" + 0.018*"like" + 0.017*"top" + 0.015*"small" + 0.012*"ordered" + 0.012*"look" + 0.010*"fabric" + 0.009*"large"
2022-02-17 17:23:31,820 : INFO : topic diff=0.059507, rho=0.180354
2022-02-17 17:23:31,822 : INFO : PROGRESS: pass 18, at document #23486/23486
2022-02-17 17:23:32,021 : INFO : optimized alpha [0.7760453, 1.5095443, 2.121868]
2022-02-17 17:23:32,022 : INFO : merging changes from 1486 documents into a model of 23486 documents
2022-02-17 17:23:32,023 : INFO : topic #0 (0.776): 0.040*"dress" + 0.025*"fit" + 0.025*"wear" + 0.024*"jean" + 0.023*"love" + 0.022*"pant" + 0.022*"perfect" + 0.018*"comfortable" + 0.015*"perfectly" + 0.015*"great"
2022-02-17 17:23:32,024 : INFO : topic #1 (1.510): 0.034*"color" + 0.030*"love" + 0.023*"great" + 0.017*"top" + 0.017*"soft" + 0.015*"look" + 0.014*"sweater" + 0.013*"wear" + 0.012*"shirt" + 0.009*"like"
2022-02-17 17:23:32,025 : INFO : topic #2 (2.122)

2022-02-17 17:23:34,204 : INFO : topic #1 (1.537): 0.034*"color" + 0.030*"love" + 0.023*"great" + 0.017*"soft" + 0.017*"top" + 0.015*"look" + 0.014*"sweater" + 0.013*"wear" + 0.012*"shirt" + 0.009*"like"
2022-02-17 17:23:34,204 : INFO : topic #2 (2.130): 0.029*"size" + 0.026*"dress" + 0.020*"fit" + 0.018*"like" + 0.016*"top" + 0.015*"small" + 0.013*"ordered" + 0.012*"look" + 0.010*"fabric" + 0.009*"large"
2022-02-17 17:23:34,205 : INFO : topic diff=0.060297, rho=0.177491
2022-02-17 17:23:34,207 : INFO : PROGRESS: pass 19, at document #18000/23486
2022-02-17 17:23:34,468 : INFO : optimized alpha [0.7917171, 1.5401301, 2.135608]
2022-02-17 17:23:34,469 : INFO : merging changes from 2000 documents into a model of 23486 documents
2022-02-17 17:23:34,470 : INFO : topic #0 (0.792): 0.040*"dress" + 0.026*"fit" + 0.025*"wear" + 0.024*"jean" + 0.023*"pant" + 0.023*"love" + 0.022*"perfect" + 0.018*"comfortable" + 0.015*"great" + 0.014*"perfectly"
2022-02-17 17:23:34,470 : INFO : topic #1 (1.540)

In [17]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-02-17 17:25:13,223 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-02-17 17:25:13,231 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-02-17 17:25:13,239 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-02-17 17:25:13,247 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-02-17 17:25:13,255 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-02-17 17:25:13,263 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-02-17 17:25:13,270 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-02-17 17:25:13,277 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-02-17 17:25:13,285 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-02-17 17:25:13,292 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-02-17 17:25:13,300 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-02-17 17:25:13

Average topic coherence: -2.9045.
[([(0.034008235, 'color'),
   (0.029812891, 'love'),
   (0.022865191, 'great'),
   (0.016971521, 'soft'),
   (0.016919194, 'top'),
   (0.01557296, 'look'),
   (0.01443356, 'sweater'),
   (0.012919248, 'wear'),
   (0.012204834, 'shirt'),
   (0.008913008, 'like'),
   (0.008859497, 'fabric'),
   (0.008856967, 'bought'),
   (0.008678984, 'black'),
   (0.008543164, 'fit'),
   (0.008387015, 'sleeve'),
   (0.008260591, 'well'),
   (0.008188984, 'cute'),
   (0.007813551, 'white'),
   (0.0077281087, 'perfect'),
   (0.007241097, 'true')],
  -2.0032589425426255),
 ([(0.028954165, 'size'),
   (0.025011756, 'dress'),
   (0.019496072, 'fit'),
   (0.017701797, 'like'),
   (0.016588923, 'top'),
   (0.015848693, 'small'),
   (0.012365411, 'ordered'),
   (0.011982658, 'look'),
   (0.010400236, 'fabric'),
   (0.009573591, 'large'),
   (0.008751002, 'really'),
   (0.008663023, 'back'),
   (0.008287796, 'petite'),
   (0.0077025453, 'little'),
   (0.007639263, 'waist'),
   

In [18]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
vis

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.202503  0.138081       1        1  50.982143
1      0.043402 -0.214017       2        1  32.886257
0     -0.245905  0.075936       3        1  16.131601, topic_info=           Term          Freq         Total Category  logprob  loglift
8         dress  11336.000000  11336.000000  Default  30.0000  30.0000
90        color   6964.000000   6964.000000  Default  29.0000  29.0000
16         love   9065.000000   9065.000000  Default  28.0000  28.0000
295        jean   2618.000000   2618.000000  Default  27.0000  27.0000
70        great   6058.000000   6058.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
6        bought   1029.388478   3044.254307   Topic3  -4.5327   0.7401
415     wearing    672.460141   1526.538864   Topic3  -4.9585   1.0046
78   flattering    888.525953   3551.364197   Topic3  -4.6798   0.4389
232         got    663.931078   2378.944087   Topic3  -4.9712   0.5482
61         work    610.999049   2523.614823   Topic3  -5.0543   0.4060

[225 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
0         1  0.121506       absolutely
0         2  0.060114       absolutely
0         3  0.817290       absolutely
556       2  0.023055  absolutely_love
556       3  0.978178  absolutely_love
...     ...       ...              ...
131       2  0.017168                x
152       1  0.998145               xx
189       2  0.126120             year
189       3  0.872923             year
1229      2  0.995939           yellow

[331 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/